In [1]:
import os
import threading
import time

import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import utils

import vrep
from hexapod import Hexapod, connect

from image_train import ImageTrain
from image_train import train_set_percentage
from image_train import test_robot

In [2]:
train = ImageTrain()

In [3]:
label_row_1, data_row_1 = train.get_data('./gen_image/')

{==========>......................................................................} 13% 

{===============================>.................................................} 38% 

{======================================================>..........................} 68% 

{=============================================================================>...} 97% 

{================================================================================>} 100% 

In [4]:
label_tmp_1 = np.argmax(label_row_1, axis=1)
label_tmp_1[np.where(label_tmp_1 == 2)[0]]=1
label_tmp_1[np.where(label_tmp_1 == 3)[0]]=1
label_tmp_1[np.where(label_tmp_1 == 5)[0]]=1
label_tmp_1[np.where(label_tmp_1 == 6)[0]]=1

In [6]:
def rechoice_j(label, data=None, num=None,shuffle=True):
    assert(num!=None)
#     label = np.argmax(label, axis=1)
    con_list=[]
    data_list=[0,1,2,3,4,5,6]
    for i,element in enumerate(num):
        con_list.append(np.random.choice(np.where(label == data_list[i])[0],element))
    result = np.concatenate(con_list)
    if shuffle:
        np.random.shuffle(result)
    label = label[result]
#     label = utils.to_categorical(label, num_classes=num_classes)
    if data is None:
        return label
    data = data[result]
    return label, data

In [29]:
data_tmp_1=data
train_label_1,train_data_1=rechoice_j(label_tmp_1,data_tmp_1,[6600,6600])
train_label_1=utils.to_categorical(train_label_1, num_classes=7)

NameError: name 'data' is not defined

In [15]:
train_set_percentage(train_label_1)

2:0 3:0 5:0 6:0 0:6601 1:13442 4:0


In [18]:
model=train.build_model(train_data_1,train_label_1)
model.fit(train_data, train_label, batch_size=32, epochs=20,
                  validation_split=0.3)

Train on 9240 samples, validate on 3960 samples
Epoch 1/20
9240/9240 [==============================] - 17s 2ms/step - loss: 1.5750 - mean_absolute_error: 0.3383 - mean_squared_error: 0.2117 - acc: 0.7392 - val_loss: 0.3241 - val_mean_absolute_error: 0.2375 - val_mean_squared_error: 0.0953 - val_acc: 0.9056
Epoch 2/20
9240/9240 [==============================] - 10s 1ms/step - loss: 0.3423 - mean_absolute_error: 0.2183 - mean_squared_error: 0.1046 - acc: 0.8575 - val_loss: 0.2230 - val_mean_absolute_error: 0.1510 - val_mean_squared_error: 0.0598 - val_acc: 0.9106
Epoch 3/20
9240/9240 [==============================] - 10s 1ms/step - loss: 0.2980 - mean_absolute_error: 0.1915 - mean_squared_error: 0.0891 - acc: 0.8784 - val_loss: 0.2018 - val_mean_absolute_error: 0.1380 - val_mean_squared_error: 0.0515 - val_acc: 0.9197
Epoch 4/20
9240/9240 [==============================] - 10s 1ms/step - loss: 0.2575 - mean_absolute_error: 0.1644 - mean_squared_error: 0.0751 - acc: 0.8984 - val_loss: 

In [19]:
model.save('turn_straight_cla.h5')

In [35]:
test_robot('turn_straight_cla.h5')

TypeError: test_robot() takes 0 positional arguments but 1 was given

In [7]:
_,data_tmp_2=rechoice_j(label_tmp_1,data_row_1,[0,13000])
label_tmp_2=np.zeros((data_tmp_2.shape[0],2))
label_tmp_2[:,1]=1

In [9]:
label_row_3,data_row_3=train.get_data_range('./image/archive-ahead-train/',987,1000)
print(label_row_3.shape)
label_tmp_3=np.zeros((data_row_3.shape[0],2))
label_tmp_3[:,0]=1

train_label_2_row = np.concatenate([label_tmp_2,label_tmp_3])
train_data_2_row = np.concatenate([data_tmp_2,data_row_3])

print(train_label_2_row.shape,train_data_2_row.shape)

(2372, 7)
(15372, 2) (15372, 64, 64, 1)


In [11]:
t_label= np.argmax(train_label_2_row, axis=1)
train_label_2_tmp,train_data_2=rechoice_j(t_label,train_data_2_row,[2300,2300])
train_label_2 = utils.to_categorical(train_label_2_tmp, num_classes=2)

model_2=train.build_model(train_data_2,train_label_2)
history=model_2.fit(train_data_2,train_label_2,batch_size=32, epochs=50,
                  validation_split=0.2)

Train on 3680 samples, validate on 920 samples
Epoch 1/50
3680/3680 [==============================] - 2s 601us/step - loss: 7.9243 - mean_absolute_error: 0.4997 - mean_squared_error: 0.4985 - acc: 0.5000 - val_loss: 8.0590 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.5000 - val_acc: 0.5000
Epoch 2/50
3680/3680 [==============================] - 2s 486us/step - loss: 3.0554 - mean_absolute_error: 0.3302 - mean_squared_error: 0.2741 - acc: 0.6981 - val_loss: 1.4750 - val_mean_absolute_error: 0.4197 - val_mean_squared_error: 0.3639 - val_acc: 0.5174
Epoch 3/50
3680/3680 [==============================] - 2s 488us/step - loss: 0.3684 - mean_absolute_error: 0.2291 - mean_squared_error: 0.1154 - acc: 0.8503 - val_loss: 0.2680 - val_mean_absolute_error: 0.2013 - val_mean_squared_error: 0.0801 - val_acc: 0.9457
Epoch 4/50
3680/3680 [==============================] - 2s 491us/step - loss: 0.3272 - mean_absolute_error: 0.2045 - mean_squared_error: 0.1018 - acc: 0.8489 - val_los

3680/3680 [==============================] - 2s 538us/step - loss: 0.2596 - mean_absolute_error: 0.1963 - mean_squared_error: 0.0759 - acc: 0.9386 - val_loss: 0.2288 - val_mean_absolute_error: 0.1772 - val_mean_squared_error: 0.0637 - val_acc: 0.9630
Epoch 33/50
3680/3680 [==============================] - 2s 530us/step - loss: 0.2629 - mean_absolute_error: 0.1972 - mean_squared_error: 0.0772 - acc: 0.9359 - val_loss: 0.2363 - val_mean_absolute_error: 0.1837 - val_mean_squared_error: 0.0665 - val_acc: 0.9554
Epoch 34/50
3680/3680 [==============================] - 2s 506us/step - loss: 0.2535 - mean_absolute_error: 0.1915 - mean_squared_error: 0.0737 - acc: 0.9408 - val_loss: 0.2381 - val_mean_absolute_error: 0.1847 - val_mean_squared_error: 0.0672 - val_acc: 0.9522
Epoch 35/50
3680/3680 [==============================] - 2s 510us/step - loss: 0.2630 - mean_absolute_error: 0.1978 - mean_squared_error: 0.0778 - acc: 0.9280 - val_loss: 0.2200 - val_mean_absolute_error: 0.1701 - val_mean_

In [12]:
model_2.save('turn_straight_cla_v0.2.h5')